<a href="https://colab.research.google.com/github/Athanaisis/ProjetAgenceGestionSortiesDecouverte/blob/main/space_titanic_projet_google_cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Loader les données de BigQuery dans un DataFrame

In [2]:
from google.cloud import bigquery
import pandas as pd

In [6]:
# J'instancie le client BigQuery
client = bigquery.Client(project='woven-catwalk-450510-e5')

In [7]:
# J'écris la requête SQL pour charger les données
query = "SELECT * FROM train_data.train_tabl"

In [9]:
from google.colab import auth
auth.authenticate_user()

In [10]:
# Je charge les données dans un DataFrame
df = client.query(query).to_dataframe()

#Nettoyer les Données

In [12]:
df.dropna(inplace=True)

In [11]:
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,6145_01,Europa,<NA>,C/231/S,55 Cancri e,NaN,False,3478.0,10.0,0.0,105.0,2383.0,Benebah Asolipery,False
1,0052_01,Earth,False,G/6/S,TRAPPIST-1e,NaN,False,4.0,0.0,2.0,4683.0,0.0,Elaney Hubbarton,False
2,0068_01,Mars,False,E/4/S,TRAPPIST-1e,NaN,False,793.0,0.0,2.0,253.0,0.0,Cinst Binie,False
3,0202_02,Europa,False,A/2/P,55 Cancri e,NaN,False,0.0,2433.0,NaN,878.0,443.0,Vegas Embleng,True
4,0206_01,Europa,False,C/9/S,55 Cancri e,NaN,False,2.0,1720.0,12.0,1125.0,122.0,Nuson Brugashed,True


In [13]:
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
182,0017_01,Earth,False,G/0/P,TRAPPIST-1e,0.0,False,0.0,0.0,0.0,0.0,0.0,Lyde Brighttt,True
184,0179_02,Earth,False,G/26/P,TRAPPIST-1e,0.0,False,0.0,0.0,0.0,0.0,0.0,Raque Webstephrey,False
185,0193_03,Mars,False,F/41/P,TRAPPIST-1e,0.0,False,0.0,0.0,0.0,0.0,0.0,Goles Raf,True
186,0220_02,Earth,False,G/37/P,TRAPPIST-1e,0.0,False,0.0,0.0,0.0,0.0,0.0,Wena Wilsoney,True
187,0313_04,Earth,False,G/50/P,55 Cancri e,0.0,False,0.0,0.0,0.0,0.0,0.0,Hary Mckinsond,True


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6606 entries, 182 to 8691
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   6606 non-null   object 
 1   HomePlanet    6606 non-null   object 
 2   CryoSleep     6606 non-null   boolean
 3   Cabin         6606 non-null   object 
 4   Destination   6606 non-null   object 
 5   Age           6606 non-null   float64
 6   VIP           6606 non-null   boolean
 7   RoomService   6606 non-null   float64
 8   FoodCourt     6606 non-null   float64
 9   ShoppingMall  6606 non-null   float64
 10  Spa           6606 non-null   float64
 11  VRDeck        6606 non-null   float64
 12  Name          6606 non-null   object 
 13  Transported   6606 non-null   boolean
dtypes: boolean(3), float64(6), object(5)
memory usage: 916.1+ KB


In [16]:
df.isna().sum().sum()

0

#Préprocesser ma data

In [18]:
# Encodage des variables catégorielles
df = pd.get_dummies(df, columns=['HomePlanet', 'Cabin', 'Destination', 'Name'])

In [19]:
# Normalisation ou standardisation
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [20]:
df[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']] = scaler.fit_transform(df[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']])

In [21]:
# Conversion des booléens en entiers
df['CryoSleep'] = df['CryoSleep'].astype(int)
df['VIP'] = df['VIP'].astype(int)
df['Transported'] = df['Transported'].astype(int)

# 1. Instancier un Modèle

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [24]:
# Je sépare les caractéristiques et la cible
X = df.drop('Transported', axis=1)  # Remplace 'target_column' par le nom de ta colonne cible
y = df['Transported']

# Je sépare les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# J'instancie le modèle
model = RandomForestClassifier()

# 2. Entraîner le modèle

In [25]:
model.fit(X_train, y_train)

RandomForestClassifier()

# 3. Faire des prédictions

In [26]:
y_pred = model.predict(X_test)

# 4. Évaluer le modèle

In [28]:
from sklearn.metrics import classification_report, accuracy_score

In [29]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

In [30]:
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", report)

Accuracy: 0.77
Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.79      0.78       660
           1       0.78      0.75      0.77       662

    accuracy                           0.77      1322
   macro avg       0.77      0.77      0.77      1322
weighted avg       0.77      0.77      0.77      1322



print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", report)

# Uploader le modèle sur Cloud Storage

In [31]:
import joblib
from google.cloud import storage

In [32]:
# Je sauvegarde le modèle localement
joblib.dump(model, 'model.pkl')

# J'uploade le modèle sur Cloud Storage
storage_client = storage.Client()
bucket = storage_client.bucket('athanaisis_space_titanic_bucket')  # Je remplace par le nom de mon bucket
blob = bucket.blob('model.pkl')
blob.upload_from_filename('model.pkl')

#Packager le code sous forme de fonction

In [40]:
def process_and_train_model(project_id, big_query_request_sql,target_column, bucket_name,model_filename='model.pkl'):

    # 1. Authentification (nécessaire si utilisé dans un environnement local)
    from google.colab import auth
    auth.authenticate_user()

    # 2. Instancier le client BigQuery
    client = bigquery.Client(project=project_id)

    # 3. Charger les données depuis BigQuery
    dataframe = client.query(big_query_request_sql).to_dataframe()  # Exécuter la requête et charger dans un DataFrame

    # 4. Nettoyage des données
    dataframe.dropna(inplace=True)  # Suppression des lignes avec des valeurs manquantes

    # 5. Identifier et transformer les colonnes booléennes en entiers
    boolean_columns = [col for col in dataframe.columns if dataframe[col].dtype == 'bool']
    dataframe[boolean_columns] = dataframe[boolean_columns].astype(int)

    # 6. Séparer les caractéristiques et la cible
    X = dataframe.drop(target_column, axis=1)
    y = dataframe[target_column]

    # 7. Identifier les types de colonnes
    numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_columns = X.select_dtypes(include=['object']).columns.tolist()

     # 8. Import ColumnTransformer, StandardScaler and OneHotEncoder within the function
    from sklearn.preprocessing import StandardScaler, OneHotEncoder
    from sklearn.compose import ColumnTransformer
    from sklearn.pipeline import Pipeline
    from sklearn.ensemble import RandomForestClassifier # Import RandomForestClassifier as well
    from sklearn.metrics import accuracy_score, classification_report # Import necessary modules for model evaluation


    # 9. Créer un préprocesseur
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_columns),  # Normalisation
            ('cat', OneHotEncoder(), categorical_columns)   # Encodage
        ]
    )

    # 10. Créer un pipeline avec le préprocesseur et le modèle
    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
    ])

    # 11. Entraîner le modèle
    model.fit(X, y)

    # 10. Évaluation du modèle
    y_pred = model.predict(X)
    accuracy = accuracy_score(y, y_pred)
    report = classification_report(y, y_pred)

    print(f"Accuracy: {accuracy:.2f}")
    print("Classification Report:\n", report)

    # 11. Sauvegarde le modèle localement
    joblib.dump(model, model_filename)

    # 12. Uploader le modèle sur Cloud Storage
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(model_filename)
    blob.upload_from_filename(model_filename)

# def process_and_train_model(project_id, big_query_request_sql,target_column, bucket_name,model_filename='model.pkl'):

# Exemple d'utilisation
project_id = 'woven-catwalk-450510-e5'  

# Remplace par ton ID de projet
bq_query = "SELECT * FROM train_data.train_tabl"  

# Remplace par ta requête BigQuery
target_column = 'Transported'  

# Remplace par le nom de ta colonne cible
process_and_train_model(project_id, bq_query, target_column)

In [41]:
# Appel de la fonction en spécifiant un nom de fichier personnalisé
process_and_train_model('woven-catwalk-450510-e5', "SELECT * FROM train_data.train_tabl",'Transported', 'athanaisis_space_titanic_bucket',model_filename='modèle_version2.pkl')

Accuracy: 1.00
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      3279
         1.0       1.00      1.00      1.00      3327

    accuracy                           1.00      6606
   macro avg       1.00      1.00      1.00      6606
weighted avg       1.00      1.00      1.00      6606



In [42]:
! apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


In [43]:
! git config --global user.name "Athanaisis"
! git config --global user.email "lavril_anasthasia@yahoo.fr"

In [45]:
!mkdir /content/space_titanic_project
!cd /content/space_titanic_project && git init
!cp /content/mon_script.py /content/space_titanic_project/
%cd /content/space_titanic_project
!git add .
!git commit -m "Premier commit depuis Colab"
!git remote add origin https://github.com/Athanaisis/space_titanic.git
!git push -u origin master

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/space_titanic_project/.git/
cp: cannot stat '/content/mon_script.py': No such file or directory
/content/space_titanic_project
On branch master

Initial commit

nothing to commit (create/copy files and use "git add" to track)
error: src refspec master does not match any
error: failed to push some refs to 'https://github.com/Athanaisis/space_titanic.git'


In [46]:
with open('/content/space_titanic_project/mon_script.py', 'w') as f:
    f.write('print("Hello World")')